In [1]:
import os
import csv
import time
import shutil
import requests
import pandas as pd
from splinter import Browser

In [3]:
links = pd.read_csv("links_filtered_final.csv")
links.head(15)

,movieId,imdbId,tmdbId
0,307,108394,108
1,481,107302,10909
2,1091,98627,8491
3,1257,88794,13667
4,1449,118111,16448
5,1590,119081,8413
6,1591,120177,10336
7,2134,90305,11814
8,2478,92086,8388
9,2840,145531,10307


In [4]:
#activates google chrome for automation
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

In [49]:
def pairwise(it):
    it = iter(it)
    while True:
        yield next(it), next(it)

In [94]:
metadata = {"id": [],
          "title": [],
          "overview": [],
          "crew": [],
          "cast": [],
          "genres": [],
          "language": [],
          "runtime": [],
          "budget": [],
          "revenue": []}

In [96]:
missing = []

In [97]:
def tmdb_scrape(movie_df, start):
    start_scrape = time.time()
    countdown = 8050
    for index, movie in movie_df[start:].iterrows():
        countdown -= 1
        print(f'{countdown}: left')
        start_download = time.time()
        movieId = int(movie["movieId"])
        tmdbId = int(movie["tmdbId"])
        url = f'https://www.themoviedb.org/movie/{tmdbId}'
        browser.visit(url)

        try:
            if(browser.is_element_present_by_text("Overview", wait_time=2)):

                title = browser.title.replace(" — The Movie Database (TMDb)", "")
                overview = browser.find_by_css("div[class='overview'] p").value
                crew = browser.find_by_css("li[class='profile'] p")
                people = []
                for a, b in pairwise(crew):
                    person = (a.value, b.value)
                    people.append(person)
                
                cast = browser.find_by_css("li[class='card'] p a")
                cast = [g.value for g in cast] 
                genres = browser.find_by_css("section[class='genres right_column'] li a")
                genres = [g.value for g in genres]
                info = browser.find_by_css("section[class='facts left_column'] p")
                info = [g.value[g.value.find("\n") + 1:] for g in info]
                info = info[-4:]
                lang = info[0]
                runtime = info[1]
                budget = info[2]
                revenue = info[3]

                print(movieId, title, tmdbId)
                print('\n')
                print(people)
                print('\n')
                print("Cast:", cast)
                print('\n')
                print(genres)
                print('\n')
                print(lang, runtime, budget, revenue)
                #print(info)
                print('\n')
                print(overview)
                print("_____________________________________________________")
                
                metadata["id"].append(movieId)
                metadata["title"].append(title)
                metadata["overview"].append(overview)
                metadata["crew"].append(people)
                metadata["cast"].append(cast)
                metadata["genres"].append(genres)
                metadata["language"].append(lang)
                metadata["runtime"].append(runtime)
                metadata["budget"].append(budget)
                metadata["revenue"].append(revenue)

            else:
                print(movieId, '- tag not found')
                continue

        except Exception as e:
            print(movieId, e)
            missing.append(movieId)
            continue

    scrape_time = time.time() - start_scrape
    print(f'Scrape completed in {scrape_time} seconds.')

In [ ]:
tmdb_scrape(links, 0)

In [101]:
md_df = pd.DataFrame.from_dict(metadata)

In [102]:
md_df['overview'][0]

'Julie is haunted by her grief after living through a tragic auto wreck that claimed the life of her composer husband and young daughter. Her initial reaction is to withdraw from her relationships, lock herself in her apartment and suppress her pain. But avoiding human interactions on the bustling streets of Paris proves impossible, and she eventually meets up with Olivier, an old friend who harbors a secret love for her, and who could draw her back to reality.'

In [103]:
md_df.to_csv("metadata.csv",index=False)

In [106]:
csv = pd.read_csv("metadata.csv")
csv.head(15)

,id,title,overview,crew,cast,genres,language,runtime,budget,revenue
0,307,Three Colors: Blue (1993),Julie is haunted by her grief after living thr...,"[('Krzysztof Kieślowski', 'Author, Director, S...","['Juliette Binoche', 'Benoît Régent', 'Florenc...","['DRAMA', 'MUSIC', 'MYSTERY']",French,1h 38m,-,-
1,481,Kalifornia (1993),A journalist duo go on a tour of serial killer...,"[('Dominic Sena', 'Director'), ('Tim Metcalfe'...","['', 'David Duchovny', 'Juliette Lewis', 'Mich...","['THRILLER', 'CRIME']",English,1h 57m,"$9,000,000.00","$2,395,231.00"
2,1091,Weekend at Bernie's (1989),Two friends are invited for a weekend to a lux...,"[('Ted Kotcheff', 'Director'), ('Robert Klane'...","['', 'Jonathan Silverman', 'Catherine Mary Ste...",['COMEDY'],English,1h 37m,"$6,500,000.00","$30,218,387.00"
3,1257,Better Off Dead... (1985),Lane Meyer is obsessed with his girlfriend Bet...,"[('Savage Steve Holland', 'Director, Writer')]","['', 'David Ogden Stiers', 'Kim Darby', 'Demia...",['COMEDY'],English,1h 37m,-,"$10,297,601.00"
4,1449,Waiting for Guffman (1996),"Corky St. Clair is a director, actor and dance...","[('Christopher Guest', 'Director, Writer'), ('...","['', 'Eugene Levy', 'Fred Willard', ""Catherine...","['COMEDY', 'MUSIC']",English,1h 24m,"$4,000,000.00",-
5,1590,Event Horizon (1997),In 2047 a group of astronauts are sent to inve...,"[('Paul W.S. Anderson', 'Director'), ('Philip ...","['', 'Sam Neill', 'Kathleen Quinlan', 'Joely R...","['HORROR', 'SCIENCE FICTION', 'MYSTERY']",English,1h 36m,"$60,000,000.00","$26,673,242.00"
6,1591,Spawn (1997),After being murdered by corrupt colleagues in ...,"[('Todd McFarlane', 'Author'), ('Mark A.Z. Dip...","['', 'Martin Sheen', 'John Leguizamo', 'Theres...","['ACTION', 'ADVENTURE', 'FANTASY', 'HORROR', '...",English,1h 36m,"$40,000,000.00","$87,840,042.00"
7,2134,Weird Science (1985),"Two unpopular teenagers, Gary and Wyatt, fail ...","[('John Hughes', 'Director, Screenplay')]","['Anthony Michael Hall', 'Kelly LeBrock', 'Ila...","['COMEDY', 'ROMANCE', 'SCIENCE FICTION']",English,1h 34m,"$7,500,000.00","$38,934,048.00"
8,2478,¡Three Amigos! (1986),Three unemployed actors accept an invitation t...,"[('John Landis', 'Director'), ('Lorne Michaels...","['', 'Steve Martin', 'Martin Short', 'Patrice ...","['COMEDY', 'WESTERN']",English,1h 42m,"$25,000,000.00",-
9,2840,Stigmata (1999),A young woman with no strong religious beliefs...,"[('Rupert Wainwright', 'Director'), ('Rick Ram...","['', 'Gabriel Byrne', 'Jonathan Pryce', 'Nia L...","['HORROR', 'THRILLER']",English,1h 43m,"$29,000,000.00","$50,041,732.00"


In [107]:
missing

[]

In [118]:
import ast

In [121]:
h = csv['crew'][0]
i = ast.literal_eval(h)
i[0][0]

'Krzysztof Kieślowski'